Questão 1 :
Descrição da Atividade
Desenvolva um modelo preditivo capaz de classificar a cada dia a probabilidade de que a ação da empresa AMZN negociada na Nasdaq alcance nos próximos 10 dias uma valorização de no mínimo 4% e que não desvalorize mais que 2,5%.
A variável dependente já está calculada para facilitar o desenvolvimento e ela representa a condição de atingimento de resultados citada acima.
Para essa atividade disponibilizaremos um dataset com os dados diários de 10 anos de negociação segmentado em Treino (80%) e Teste (20%).
Utilize o dataset de teste para calcular as principais métricas de um modelo de classificação binário, considerando também que as classes são desbalanceadas, portanto deve ser escolhida além das métricas tradicionais métricas que considerem a condição de classes desbalanceadas.
O objetivo dessa atividade não é encontrar respostas certas, mas sim avaliar o pipeline de desenvolvimento de um modelo de Machine Learning e sua criatividade em achar caminho para solucionar um problema, portanto é importante que você descreva cada etapa do desenvolvimento explicando o porquê dos métodos utilizados.
Documente no notebook cada etapa executada de forma que facilite a sua apresentação na etapa de avaliação e apresentação.
Sobre o DataSet
O dataset é formado por dados referente aos níveis de preços em que a ação foi negociada diariamente, sendo composto pelo candle (diário) e demais fields fornecidas no terminal Bloomberg.


In [3]:
## MONTANDO O DRIVE DDO GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install sweetviz

     |████████████████████████████████| 15.1MB 211kB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


# Importando bibliotecas que serão utilizadas no código

In [1]:
# Importando o Pandas
import pandas as pd
import sweetviz as sv
from sklearn import preprocessing
# Importando as bibliotecas e funções utilizadas no modelo
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [2]:
# Lendo uma base de dados no formato CSV
# o parâmetro sep é usado para definir qual o separador entre os dados
# o parâmetro head informa em qual linha estão as descrições das colunas
# Caso não haja nomes para as colunas o pandas dará um número para cada atributo da base.

arquivo = '/content/drive/MyDrive/Callik/AMZN ATUALIZADO (1).csv'
dataset = pd.read_csv(arquivo, sep=';', header=0 )

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Callik/AMZN ATUALIZADO (1).csv'

# Verificando valores nulos

In [ ]:
dataset.isnull().sum()

date                    6
ticker                  6
ALPHA_OVERRIDABLE       6
ASSET_CVRG_RATIO        6
BB_1YR_DEFAULT_PROB     6
                       ..
VOLATILITY_90D          6
VWAP_VOLUME             6
WACC_COST_DEBT          6
WACC_COST_EQUITY        6
Variavel_resposta      34
Length: 109, dtype: int64

# Deletando valores nulos

In [ ]:
dataset = dataset.dropna(axis=0,  inplace=False)


In [ ]:
dataset.isnull().sum()

date                   0
ticker                 0
ALPHA_OVERRIDABLE      0
ASSET_CVRG_RATIO       0
BB_1YR_DEFAULT_PROB    0
                      ..
VOLATILITY_90D         0
VWAP_VOLUME            0
WACC_COST_DEBT         0
WACC_COST_EQUITY       0
Variavel_resposta      0
Length: 109, dtype: int64

# Vrificando número de linhas e colunas do dataset

In [ ]:
dataset.shape

(2853, 109)

# Verificando informações do dataset

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2853 entries, 28 to 2880
Columns: 109 entries, date to Variavel_resposta
dtypes: float64(10), object(99)
memory usage: 2.4+ MB


# Visualizando o dataset com o comando .head() que nos trás as 5 primeiras linhas 

In [ ]:
dataset.head()

,date,ticker,ALPHA_OVERRIDABLE,ASSET_CVRG_RATIO,BB_1YR_DEFAULT_PROB,BB_2Y_DEFAULT_PROB,BB_3Y_DEFAULT_PROB,BB_4Y_DEFAULT_PROB,BB_5Y_DEFAULT_PROB,BEST_ANALYST_RATING,BEST_BPS,BEST_CPS,BEST_CUR_EV_TO_EBITDA,BEST_EBIT,BEST_EBITDA,BEST_EPS,BEST_EPS_GAAP,BEST_EPS_MEDIAN,BEST_EPS_NUMEST,BEST_NET_DEBT,BEST_NET_INCOME,BEST_OPP,BEST_PEG_RATIO,BEST_PE_RATIO_GAAP,BEST_ROE,BEST_SALES,BEST_TARGET_PRICE,BETA_ADJ_OVERRIDABLE,BETA_STD_DEV_ERR_OVERRIDABLE,BLOOMBERG_ISSUER_DEFAULT_SCORE,CAPITAL_EXPEND,CAP_EXPEND_RATIO,CASH_RATIO,CFO_TO_AVG_CURRENT_LIABILITIES,CF_CASH_FROM_OPER,CF_FREE_CASH_FLOW,COUNTRY_RISK_DIVIDEND_YIELD,COUNTRY_RISK_GROWTH_RATE,COUNTRY_RISK_MARKET_RETURN,COUNTRY_RISK_PAYOUT_RATIO,...,PX_BID,PX_ERN_RATIO_WITH_LOW_CLOS_PX,PX_HIGH,PX_LAST,PX_LOW,PX_MID,PX_OFFICIAL_CLOSE,PX_OPEN,PX_TO_CASH_FLOW,PX_TO_EBITDA,PX_TO_EPS_BEFORE_ABNORMAL_ITEMS,PX_TO_FREE_CASH_FLOW,PX_TO_SALES_RATIO,PX_TO_TANG_BV_PER_SH,PX_VOLUME,QUICK_RATIO,REL_SHR_PX_MOMENTUM,RISK_PREMIUM,RSK_BB_IMPLIED_CDS_SPREAD,SALES_GROWTH,SALES_REV_TURN,SHORT_AND_LONG_TERM_DEBT,T12M_DIL_PE_CONT_OPS,TOTAL_DEBT_TO_EV,TOT_ANALYST_REC,TOT_BUY_REC,TOT_DEBT_TO_COM_EQY,TOT_DEBT_TO_TOT_ASSET,TOT_HOLD_REC,TOT_SELL_REC,VAR_SWAP_12M_LV,VOLATILITY_20D,VOLATILITY_260D,VOLATILITY_30D,VOLATILITY_360D,VOLATILITY_90D,VWAP_VOLUME,WACC_COST_DEBT,WACC_COST_EQUITY,Variavel_resposta
28,12/02/2010 00:00,AMZN US Equity,"0,1841","13,6285","0,000030138","0,000721139","0,002394981","0,004595436","0,007077496","4,306",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"159,229","1,0642","0,0579",2.0,-137,"19,0511","0,8645","0,5437",2610,2473,"2,0212","10,7221","10,9782","29,4667",...,"119,66","56,9978","119,94","119,66","117,5","119,735",0,"118,99","15,9471","34,2289","55,7525","18,0086","2,1203","13,2063",8075533.0,"0,9986","27,655","6,6138",19.0,"41,9899",9519,393,"56,9978","0,0073",36.0,25.0,"7,4757","2,8451",10.0,1.0,0,"33,915","45,736","31,347","62,273","51,194",8024738.0,"3,2031","10,4506",0.0
29,16/02/2010 00:00,AMZN US Equity,"0,1837","13,6285","0,000031043","0,000732654","0,002420904","0,004634488","0,007125953","4,306",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"159,229","1,0608","0,058",2.0,-137,"19,0511","0,8645","0,5437",2610,2473,"1,9829","10,1503","10,7191","29,471",...,"117,36","55,9832","120,5","117,53","117,18","117,445",0,"120,06","15,6632","33,6196","54,7601","17,6881","2,0826","12,9712",8938494.0,"0,9986","28,96","6,6138",21.0,"41,9899",9519,393,"55,9832","0,0073",36.0,25.0,"7,4757","2,8451",10.0,1.0,0,"34,224","45,577","31,623","62,226","51,053",8897440.0,"3,2031","10,4506",1.0
30,17/02/2010 00:00,AMZN US Equity,"0,1804","13,6285","0,000031587","0,000739574","0,002436533","0,004658125","0,007155382","4,306",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"159,108","1,0606","0,058",2.0,-137,"19,0511","0,8645","0,5437",2610,2473,"1,9744","10,1777","10,6193","29,4753",...,"116,29","55,402","117,13","116,31","115,55","116,3",0,"117,075","15,5006","33,2706","54,1917","17,5044","2,061","12,8366",8947449.0,"0,9986","29,386","6,6138",21.0,"41,9899",9519,393,"55,402","0,0073",36.0,25.0,"7,4757","2,8451",10.0,1.0,0,"34,085","45,485","31,503","62,218","51,066",8902406.0,"3,2031","10,4506",1.0
31,18/02/2010 00:00,AMZN US Equity,"0,1816","13,6285","0,000030804","0,000729587","0,002413965","0,004623984","0,007112868","4,306",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"159,108","1,0607","0,0579",2.0,-137,"19,0511","0,8645","0,5437",2610,2473,"1,9651","10,2192","10,5775","29,459",...,"118,08","56,2452","118,51","118,08","114,82","118,125",0,"115,839","15,7365","33,777","55,0164","17,7708","2,0923","13,0319",9807497.0,"0,9986","30,361","6,6138",21.0,"41,9899",9519,393,"56,2452","0,0073",36.0,25.0,"7,4757","2,8451",10.0,1.0,0,"34,61","45,229","31,787","62,219","51,111",9623213.0,"3,2031","10,4506",1.0
32,19/02/2010 00:00,AMZN US Equity,"0,1799","13,6285","0,000031048","0,000732714","0,002421041","0,004634696","0,007126212","4,306",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"159,108","1,0607","0,0578",2.0,-137,"19,0511","0,8645","0,5437",2610,2473,"1,965","10,2

# Verificando as colunas do dataset

In [ ]:
dataset.columns

Index(['date', 'ticker', 'ALPHA_OVERRIDABLE', 'ASSET_CVRG_RATIO',
       'BB_1YR_DEFAULT_PROB', 'BB_2Y_DEFAULT_PROB', 'BB_3Y_DEFAULT_PROB',
       'BB_4Y_DEFAULT_PROB', 'BB_5Y_DEFAULT_PROB', 'BEST_ANALYST_RATING',
       ...
       'VAR_SWAP_12M_LV', 'VOLATILITY_20D', 'VOLATILITY_260D',
       'VOLATILITY_30D', 'VOLATILITY_360D', 'VOLATILITY_90D', 'VWAP_VOLUME',
       'WACC_COST_DEBT', 'WACC_COST_EQUITY', 'Variavel_resposta'],
      dtype='object', length=109)

# Verificando valores únicos da variável resposta

In [ ]:
dataset.Variavel_resposta.unique()

array([0., 1.])

# Agrupando a variável resposta

In [ ]:
dataset.groupby('Variavel_resposta')['Variavel_resposta'].count()

Variavel_resposta
0.0    1909
1.0     944
Name: Variavel_resposta, dtype: int64

# Balanceando as classses

In [ ]:
data2 = dataset[dataset.Variavel_resposta==0.0].sample(944)


In [ ]:
dataset = data2.append(dataset[dataset.Variavel_resposta==1.0].sample(944))

In [ ]:
dataset.Variavel_resposta.value_counts()

1.0    944
0.0    944
Name: Variavel_resposta, dtype: int64

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1888 entries, 1656 to 2069
Columns: 109 entries, date to Variavel_resposta
dtypes: float64(10), object(99)
memory usage: 1.6+ MB


# Verificando valores únicos da coluna ticker e deletando a mesma por só conter um valor único

In [ ]:
dataset.ticker.unique()

array(['AMZN US Equity'], dtype=object)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1888 entries, 1656 to 2069
Columns: 109 entries, date to Variavel_resposta
dtypes: float64(10), object(99)
memory usage: 1.6+ MB


In [ ]:
dataset.drop(['ticker'], axis=1, inplace=True)

In [ ]:
dataset.head()

,date,ALPHA_OVERRIDABLE,ASSET_CVRG_RATIO,BB_1YR_DEFAULT_PROB,BB_2Y_DEFAULT_PROB,BB_3Y_DEFAULT_PROB,BB_4Y_DEFAULT_PROB,BB_5Y_DEFAULT_PROB,BEST_ANALYST_RATING,BEST_BPS,BEST_CPS,BEST_CUR_EV_TO_EBITDA,BEST_EBIT,BEST_EBITDA,BEST_EPS,BEST_EPS_GAAP,BEST_EPS_MEDIAN,BEST_EPS_NUMEST,BEST_NET_DEBT,BEST_NET_INCOME,BEST_OPP,BEST_PEG_RATIO,BEST_PE_RATIO_GAAP,BEST_ROE,BEST_SALES,BEST_TARGET_PRICE,BETA_ADJ_OVERRIDABLE,BETA_STD_DEV_ERR_OVERRIDABLE,BLOOMBERG_ISSUER_DEFAULT_SCORE,CAPITAL_EXPEND,CAP_EXPEND_RATIO,CASH_RATIO,CFO_TO_AVG_CURRENT_LIABILITIES,CF_CASH_FROM_OPER,CF_FREE_CASH_FLOW,COUNTRY_RISK_DIVIDEND_YIELD,COUNTRY_RISK_GROWTH_RATE,COUNTRY_RISK_MARKET_RETURN,COUNTRY_RISK_PAYOUT_RATIO,COUNTRY_RISK_PREMIUM,...,PX_BID,PX_ERN_RATIO_WITH_LOW_CLOS_PX,PX_HIGH,PX_LAST,PX_LOW,PX_MID,PX_OFFICIAL_CLOSE,PX_OPEN,PX_TO_CASH_FLOW,PX_TO_EBITDA,PX_TO_EPS_BEFORE_ABNORMAL_ITEMS,PX_TO_FREE_CASH_FLOW,PX_TO_SALES_RATIO,PX_TO_TANG_BV_PER_SH,PX_VOLUME,QUICK_RATIO,REL_SHR_PX_MOMENTUM,RISK_PREMIUM,RSK_BB_IMPLIED_CDS_SPREAD,SALES_GROWTH,SALES_REV_TURN,SHORT_AND_LONG_TERM_DEBT,T12M_DIL_PE_CONT_OPS,TOTAL_DEBT_TO_EV,TOT_ANALYST_REC,TOT_BUY_REC,TOT_DEBT_TO_COM_EQY,TOT_DEBT_TO_TOT_ASSET,TOT_HOLD_REC,TOT_SELL_REC,VAR_SWAP_12M_LV,VOLATILITY_20D,VOLATILITY_260D,VOLATILITY_30D,VOLATILITY_360D,VOLATILITY_90D,VWAP_VOLUME,WACC_COST_DEBT,WACC_COST_EQUITY,Variavel_resposta
1656,04/07/2016 00:00,"0,1581","1,9744","0,00001992","0,000551899","0,001960539","0,003877722","0,006124722","4,696",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"818,971","1,0668","0,0883",2.0,-1711,"2,0912","0,559","0,4877",3578,1867,"2,1555","10,9275","9,2976","36,1322","7,8534",...,"725,46","180,3356",728,"725,68","716,54","725,575","725,68","717,32","26,1564","31,1788","177,0089","44,5618","2,8307","26,9486",2920423.0,"0,7649","5,557","8,2725",73.0,"31,1365",30404,17860,"180,3356","0,0524",46.0,40.0,"107,9937","27,4448",6.0,0.0,"34,99338","21,022","34,874","18,655","34,903","27,212",2146754.0,"0,9112","9,7422",0.0
1133,13/06/2014 00:00,"-0,0115","4,1053","0,000005232","0,000255258","0,001148839","0,002516631","0,004303575","4,511",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"418,526","1,2661","0,0931",1.0,-1080,"-2,3167","0,4721","0,327",-2502,-3582,"2,0385","11,397","9,672","33,2243","7,0687",...,"326,11","517,8889","329,298","326,27","323,526","326,19",0,"327,05","27,8698","34,9678","509,7969","99,1288","1,9122","19,5549",2766909.0,"0,687","-22,917","9,591",69.0,"22,8438",19741,3931,"517,8889","0,0262",47.0,37.0,"38,0616","10,8101",9.0,1.0,"32,977","27,338","31,13","30,378","30,679","33,452",2749937.0,"1,4411","12,309",0.0
520,20/01/2012 00:00,"0,0702","3,7849","0,000004966","0,000265233","0,001213596","0,002677083","0,004575087","4,128",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"234,714","1,0135","0,062",1.0,-550,"7,7618","0,6429","0,3089",4269,3719,"2,153","11,1966","11,1049","26,1253","9,0803",...,"190,84","139,365","192,9","190,93","189,04","190,885",0,"190,71","22,3178","44,4956","137,3597","41,8296","1,8011","16,8522",5610542.0,"0,8155","-12,208","10,4057",34.0,"34,6231",17431,2194,"139,365","0,0307",39.0,24.0,"28,2841","8,6795",14.0,1.0,"40,36144","31,068","39,385","34,975","37,043","46,169",5571297.0,"1,4127","12,2819",0.0
2878,28/12/2020 00:00,"0,1094",\N,"0,000027971","0,000683109","0,00229477","0,004426168","0,006849478","4,893","194,114","37,533","25,882","4076,5","14131,409","10,34","7,106","10,675",14.0,"46814,5","5291,111","4372,471","1,632","83,982","20,793","119654,025","3820,008","0,8385","0,0429",2.0,\N,\N,\N,\N,\N,\N,"1,4304","13,408","8,6366","31,8155","7,7136",...,"3283,96","96,1628",3304,"3283,96","3172,69","3284,49","3283,96",3194,"29,6173","33,9342","94,3667","66,308","4,708","24,1158",5686787.0,\N,"1,912","5,4824",11.0,\N,\N,\N,"96,1628",\N,56.0,54.0,\N,\N,2.0,0.0,"38,213545","20,87","38,776","18,589","34,428","36,286",2577972.0,"0,8207","6,4171",0.0
1090,14/04/2014 00:00,"0,0443","4,1053","0,00000745","0,000306154","0,001289526","0,002746856","0,004603659","4,468",0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,"432,7

# Verificando a matriz de correlação das colunas do dataset

In [ ]:
dataset.corr()

,BEST_EPS_NUMEST,BLOOMBERG_ISSUER_DEFAULT_SCORE,PX_VOLUME,RSK_BB_IMPLIED_CDS_SPREAD,TOT_ANALYST_REC,TOT_BUY_REC,TOT_HOLD_REC,TOT_SELL_REC,VWAP_VOLUME,Variavel_resposta
BEST_EPS_NUMEST,1.000000,0.191064,0.084767,-0.201790,0.608566,0.677389,-0.593220,0.007899,-0.230639,0.002219
BLOOMBERG_ISSUER_DEFAULT_SCORE,0.191064,1.000000,-0.025503,0.445768,0.134875,0.117819,-0.054093,-0.062332,-0.058386,-0.015195
PX_VOLUME,0.084767,-0.025503,1.000000,-0.067728,-0.115969,-0.077984,-0.030705,0.288759,0.906060,0.009124
RSK_BB_IMPLIED_CDS_SPREAD,-0.201790,0.445768,-0.067728,1.000000,0.160191,0.023148,0.203032,-0.195818,-0.011656,-0.010084
TOT_ANALYST_REC,0.608566,0.134875,-0.115969,0.160191,1.000000,0.918114,-0.520165,-0.264537,-0.394880,-0.028544
TOT_BUY_REC,0.677389,0.117819,-0.077984,0.023148,0.918114,1.000000,-0.813319,-0.288794,-0.408396,-0.033959
TOT_HOLD_REC,-0.593220,-0.054093,-0.030705,0.203032,-0.520165,-0.813319,1.000000,0.132097,0.279575,0.026379
TOT_SELL_REC,0.007899,-0.062332,0.288759,-0.195818,-0.264537,-0.288794,0.132097,1.000000,0.328663,0.054393
VWAP_VOLUME,-0.230639,-0.058386,0.906060,-0.011656,-0.394880,-0.408396,0.279575,0.328663,1.000000,0.013056
Variavel_resposta,0.002219,-0.015195,0.009124,-0.010084,-0.028544,-0.033959,0.026379,0.054393,0.013056,1.000000


## Visualização através do SWEETIZ, é gerado um html com informações das estatísticas do dataset 

In [ ]:
insights  = sv.analyze(dataset) # cria o reporte e chama de insights
insights.show_html()


                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# Transforma colunas categóricas em numéricas para serem aceitas pelo algorítimo

In [ ]:
le = preprocessing.LabelEncoder()

for i in range(0,dataset.shape[1]):
    if dataset.dtypes[i] in ( 'object','float64'):
        dataset[dataset.columns[i]] = le.fit_transform(dataset[dataset.columns[i]])

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1888 entries, 1656 to 2069
Columns: 108 entries, date to Variavel_resposta
dtypes: int64(108)
memory usage: 1.6 MB


# PIPELINE

In [ ]:
pipe_random_forest = Pipeline([
        ('scl', StandardScaler()),
        ('clf', RandomForestClassifier())
        ]
)

In [ ]:
pipe_svm = Pipeline([
        ('scl', StandardScaler()),
        ('clf', svm.SVC())
        ]
)

In [ ]:
pipe_knn = Pipeline([
        ('scl', StandardScaler()),
        ('clf', KNeighborsClassifier())
        ]
)

# **GridSearch**

O GridSearchCV é uma ferramenta usada para automatizar o processo de ajuste dos parâmetros de um algoritmo, pois ele fará de maneira sistemática diversas combinações dos parâmetros e depois de avaliá-los os armazenará em um único objeto. O GridSearchCV é um módulo do ScikitLearn e é amplamente usado para automatizar grande parte do processo de tuning.

# Valores para Grid
No código abaixo criamos uma lista para alimentar as grids que vão ser utilizadas no parâmetro do GridSearch para testar as combinações.

In [ ]:
valores = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
grid_params_rf = [{
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': valores,
    'clf__max_depth': valores,
    'clf__min_samples_split': valores[1:]
}]

In [ ]:
grid_params_svm = [{
    'clf__kernel': ['linear', 'rbf'], 
    'clf__C': valores
}]

In [ ]:
grid_params_knn = [{
    'clf__n_neighbors': valores,
}]

# Construindo GridSearch

In [ ]:
gs_rf = GridSearchCV(
    estimator=pipe_random_forest,
    param_grid=grid_params_rf,
    scoring='accuracy',
    verbose=100,
    cv=10 
)


In [ ]:
gs_svm = GridSearchCV(
    estimator=pipe_svm,
    param_grid=grid_params_svm,
    scoring='accuracy',
    verbose=100,
    cv=10,
)

In [ ]:
gs_knn = GridSearchCV(
    estimator=pipe_knn,
    param_grid=grid_params_knn,
    scoring='accuracy',
    verbose=100,
    cv=10,
)

In [ ]:
# Deletando a variável alvo e alimentando a mesma em Y
X_train = dataset.drop('Variavel_resposta',axis=1)
y = dataset['Variavel_resposta']

# Computando o GridSearch para Random Forest
O método fit() é utilizado para treinar o algorítimo e verificar seu desempenho.

In [ ]:
gs_rf.fit(X_train,y)

A saída de streaming foi truncada nas últimas 5000 linhas.
[CV]  clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4, score=0.804, total=   1.1s
[CV] clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4 
[CV]  clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4, score=0.804, total=   1.1s
[CV] clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4 
[CV]  clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4, score=0.788, total=   1.1s
[CV] clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4 
[CV]  clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4, score=0.815, total=   1.1s
[CV] clf__criterion=entropy, clf__max_depth=8, clf__min_samples_leaf=3, clf__min_samples_split=4 
[CV]  clf__criterion=entropy, clf__max_depth=

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scl',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                

In [ ]:
# Melhores parametros e scoring
print('Melhores parâmetros: %s' % gs_rf.best_params_)
print('Melhores Acurácia: %.3f' % gs_rf.best_score_)

Melhores parâmetros: {'clf__criterion': 'gini', 'clf__max_depth': 10, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 3}
Melhores Acurácia: 0.838


# Computando o GridSearch para SVM

In [ ]:
gs_svm.fit(X_train,y)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] clf__C=1, clf__kernel=linear ....................................
[CV] ........ clf__C=1, clf__kernel=linear, score=0.725, total=   0.6s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[CV] clf__C=1, clf__kernel=linear ....................................
[CV] ........ clf__C=1, clf__kernel=linear, score=0.693, total=   0.6s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s
[CV] clf__C=1, clf__kernel=linear ....................................
[CV] ........ clf__C=1, clf__kernel=linear, score=0.725, total=   0.6s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.8s remaining:    0.0s
[CV] clf__C=1, clf__kernel=linear ....................................
[CV] ........ clf__C=1, clf__kernel=linear, score=0.667, total=   0.7s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elap

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scl',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        SVC(C=1.0, break_ties=False,
                                            cache_size=200, class_weight=None,
                                            coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='scale',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, 

In [ ]:
# Melhores parametros e scoring
print('Melhores parâmetros: %s' % gs_svm.best_params_)
print('Melhores Acurácia: %.3f' % gs_svm.best_score_)

Melhores parâmetros: {'clf__C': 10, 'clf__kernel': 'rbf'}
Melhores Acurácia: 0.769


# Computando o GridSearch para KNN

In [ ]:
gs_knn.fit(X_train,y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] clf__n_neighbors=1 ..............................................
[CV] .................. clf__n_neighbors=1, score=0.762, total=   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] clf__n_neighbors=1 ..............................................
[CV] .................. clf__n_neighbors=1, score=0.847, total=   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[CV] clf__n_neighbors=1 ..............................................
[CV] .................. clf__n_neighbors=1, score=0.815, total=   0.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[CV] clf__n_neighbors=1 ..............................................
[CV] .................. clf__n_neighbors=1, score=0.820, total=   0.1s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elap

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scl',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbose=False),
             

In [ ]:
# Melhores parametros e scoring
print('Melhores parâmetros: %s' % gs_knn.best_params_)
print('Melhores Acurácia: %.3f' % gs_knn.best_score_)

Melhores parâmetros: {'clf__n_neighbors': 1}
Melhores Acurácia: 0.795


# **Escolha do algorítimo de melhor desempenho**

Depois de verificar a melhor acurácia entres os três algoritmos, foi visto que o RF foi que teve o melhor desempenho, com isso foi o escolhido para ser utilizado em nosso modelo.

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_train, y)

NameError: ignored

# Métricas de Validação

As métricas de validação são utilizadas para analisar a qualidade dos modelos de Machine Learning. Ou seja, traz a informação do desempenho do modelo em dados desconhecidos.

As métricas de validação são utilizadas para analisar a qualidade dos modelos de Machine Learning. Ou seja, traz a informação do desempenho do modelo em dados desconhecidos.

Lembre-se, nos modelos de aprendizado supervisionado o objetivo é tentar estimar (prever) uma determinada variável. Por exemplo, classificar se o se a variável alvo será 0 ou 1.

Assim, analisando as diferentes métricas de validação dos modelos conseguimos analisar o poder preditivo do mesmo, antes de colocar em produção.

Já pensou fazer um modelo que consiga prever 99% dos dados e quando você entrega para seu superior ou cliente, o modelo não consegue acertar nada. Imagina o prejuízo financeiro da empresa e da sua reputação.

Existem diferentes métricas e algumas funcionam melhor para um determinado problema. Escolher uma boa métrica para avaliar o modelo é tão importante quanto escolher um bom modelo.  Nos modelos de classificação a maioria das métricas são baseadas na matriz de confusão.

OBS: Utilizo a palavra prever para ajudar no entendimento do conceito, mas a palavra correta é estimar. Os modelos de classificação estimam uma probabilidade da observação pertencer a uma classe. Quanto mais alta essa probabilidade, maior a chance da observação pertencer aquela classe.

No código abaixo temos algumas métricas para vermos o poder preditivo do nosso modelo.




In [ ]:
print (pd.crosstab(y_teste, gs_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

Predito    0    1  All
Real                  
0        230    7  237
1          6  229  235
All      236  236  472 


In [ ]:
print (metrics.classification_report(y_teste,gs_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       237
           1       0.97      0.97      0.97       235

    accuracy                           0.97       472
   macro avg       0.97      0.97      0.97       472
weighted avg       0.97      0.97      0.97       472



# COMUNICAÇÃO DOS RESULTADOS
Persistindo o modelo de Machine Learning para o disco.
Persistir o modelo para o disco, nada mais é de tirar da memória o modelo e coloca no seu disco físico, para com isso ser utilizado da forma que você quiser, ser implemntado em um sistema de terceiros por exemplo.

In [ ]:
# Persistindo o melhor modelo em disco.
joblib.dump(gs_rf, 'model.pkl')

['model.pkl']

# Carregando o modelo a partir do disco para a memória.
Lendo o modelo que está no disco

In [ ]:
model = joblib.load('model.pkl')

In [ ]:
print("Atributos do Modelo:\n\nClasses:{}\nEstimator:{}".format(model.classes_,model.estimator))

Atributos do Modelo:

Classes:[0 1]
Estimator:Pipeline(memory=None,
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
  